<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install \
transformers==4.38.2 \
sentence-transformers==2.6.1 \
faiss-cpu \
torch


In [ ]:
import os

os.makedirs("docs", exist_ok=True)


In [ ]:
with open("docs/register.md", "w", encoding="utf-8") as f:
    f.write("""
# التسجيل في منصة عون

## خطوات التسجيل
- الدخول على موقع عون
- إنشاء حساب جديد
- إدخال البيانات الأساسية
- تأكيد رقم الهاتف

## ملاحظات
- يجب إدخال بيانات صحيحة
- رقم الهاتف إلزامي
""")


In [ ]:
with open("docs/case_submit.md", "w", encoding="utf-8") as f:
    f.write("""
# رفع حالة في منصة عون

## خطوات رفع الحالة
- تسجيل الدخول إلى الحساب
- اختيار رفع حالة جديدة
- إدخال بيانات الأسرة
- إرفاق المستندات المطلوبة
- إرسال الطلب للمراجعة

## ملاحظات
- يجب إرفاق مستندات واضحة
- الطلب يخضع للمراجعة
""")


In [ ]:
with open("docs/terms.md", "w", encoding="utf-8") as f:
    f.write("""
# شروط الاستفادة من منصة عون

- أن تكون البيانات صحيحة
- أن تكون الحالة حقيقية
- تقديم المستندات المطلوبة
- قبول سياسة الاستخدام
""")


In [ ]:
with open("docs/faq.md", "w", encoding="utf-8") as f:
    f.write("""
# الأسئلة الشائعة

## هل التسجيل مجاني؟
نعم، التسجيل مجاني تمامًا.

## هل يمكن تعديل البيانات؟
نعم، يمكن تعديل البيانات قبل إرسال الطلب.
""")


In [ ]:
!ls docs


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model_embed = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

texts = []
with open("docs/register.md", encoding="utf-8") as f:
    texts.append(f.read())

embeddings = model_embed.encode(texts)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


In [ ]:
pipe = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct",
    max_new_tokens=150,
    do_sample=False,
    temperature=None,
    top_p=None,
    top_k=None
)


In [ ]:
print(pipe("اشرح خطوات التسجيل في منصة خيرية")[0]["generated_text"])


In [ ]:
def rag_arabic(question):
    # البحث
    q_emb = model_embed.encode([question])
    _, idx = index.search(np.array(q_emb), k=1)

    context = texts[idx[0][0]]

    prompt = f"""
أجب بالعربية فقط وبوضوح.
اعتمد فقط على المعلومات التالية:

{context}

السؤال:
{question}

الإجابة:
"""

    result = pipe(prompt)
    return result[0]["generated_text"]


In [ ]:
print(rag_arabic("إزاي أسجل في منصة عون؟"))
